<a href="https://colab.research.google.com/github/FaridRash/IGNIS/blob/main/Notebooks/SNR_weighted_normalized_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""https://wp.optics.arizona.edu/alumni/wp-content/uploads/sites/113/2024/06/Jacob_Wilson_Master_s_Report_2024.pdf
maybe when you do the calculation cite that the value is taken from this source"""

'https://wp.optics.arizona.edu/alumni/wp-content/uploads/sites/113/2024/06/Jacob_Wilson_Master_s_Report_2024.pdf\nmaybe when you do the calculation cite that the value is taken from\xa0this\xa0source'

In [2]:
"""For the campi flegrei target we are considering a 200 km^2 square area (approximately 14x14km).
But the specific spots we want to investigate are the fumarole (whose vents can reach 10 metres and 165degree)
Consider those would be much smaller than our gsd, so if you consider the signal from that area it would be a
weighted sum of the signal from the fumarola (~10m, 150-160 °C) and from the sourrunding ground (GSD-10m, Ground Temperature)

But this thing of the weighted sum can be implemented later,
for now let’s see what happens with the whole campi flegrei as target,
then let’s consider the hotspot, then let’s consider the weighted sum (if necessary)"""

'For the campi flegrei target we are considering a 200 km^2 square area (approximately 14x14km).\nBut the specific spots we want to investigate are the fumarole (whose vents can reach 10 metres and 165degree)\nConsider those would be much smaller than our gsd, so if you consider the signal from that area it would be a\nweighted sum of the signal from the fumarola (~10m, 150-160 °C) and from the sourrunding ground (GSD-10m, Ground\xa0Temperature)\n\nBut this thing of the weighted sum can be implemented later,\nfor now let’s see what happens with the whole campi flegrei as target,\nthen let’s consider the hotspot, then let’s consider the weighted sum\xa0(if\xa0necessary)'

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

In [4]:
c1 = 3.7418e8
c2 = 1.4388e4
h = 6.626e-34
c = 3e8

In [5]:
def compute_gsd(H, f, p):
  return (H * p / f) **2

In [6]:
Tf = 433.15

In [7]:
def compute_wighted_snr(Tg,Tf, theta_deg, R, tau_a):
  # The area of the target and background
    Af = 10*10
    Agsd = compute_gsd(R, focal_length, pixel)
    Ag = Agsd - Af

    #Blackbody calculation
    cos_theta = np.cos(np.deg2rad(theta_deg))
    M_lambda_g = c1 / (lambda_range**5 * (np.exp(c2 / (lambda_range * Tg)) - 1))
    M_lambda_f = c1 / (lambda_range**5 * (np.exp(c2 / (lambda_range * Tf)) - 1))

    #Radiance calculation
    E_lambda_g = epsilon * M_lambda_g
    E_lambda_f = epsilon * M_lambda_f

    #Signal calculation
    S_lambda_g = ((D**2 * t * cos_theta * Ag) / (4 * R**2 * h * c)) * (lambda_range * E_lambda_g * tau_a * tau_0 * eta * delta_lambda)
    S_lambda_f = ((D**2 * t * cos_theta * Af) / (4 * R**2 * h * c)) * (lambda_range * E_lambda_f * tau_a * tau_0 * eta * delta_lambda)


    #Sum of S
    S_Ag = np.sum(S_lambda_g)
    S_Af = np.sum(S_lambda_f)

    #S' calculation
    S_prime_g = S_Ag / Ag
    S_prime_f = S_Af / Af
    S_prime = S_prime_f - S_prime_g

    #N' calculation
    N_prime = np.sqrt(S_prime_f + S_prime_g)

    #SNR' calculation
    SNR_prime = S_prime / N_prime

    #S'_lambda calculation
    S_prime_lambda_g = S_lambda_g / Ag
    S_prime_lambda_f = S_lambda_f / Af

    #S'_lambda calculation
    S_prime_lambda = S_prime_lambda_f - S_prime_lambda_g

    #SNR'_lambda
    SNR_prime_lambda = S_prime_lambda / N_prime



    plt.figure(figsize=(10, 5))
    plt.plot(lambda_range, SNR_prime_lambda, label='SNR Contribution per λ', color='blue')
    plt.xlabel('Wavelength (μm)')
    plt.ylabel('SNR Contribution')
    plt.title(f'Spectral SNR Contribution\nTg = {Tg}K, Tf = {Tf}K, θ = {theta_deg}°, R = {R/1000:.0f}km, τₐ = {tau_a}, Ag = {Ag:.2f}m, Af = {Af:.0f}m' )
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

    print(f"Total SNR' at Tg = {Tg} K, Tf = {Tf} K, θ = {theta_deg}°, R = {R/1000:.0f} km, τₐ = {tau_a}: {SNR_prime:.2f}")

In [8]:
epsilon = 0.95
tau_0 = 0.89
eta = 0.7
t = 0.01
focal_length = 72.8 / 1000
pixel = 12e-6
F_number = 1.05
D = (focal_length / F_number)

lambda_range = np.arange(8, 12.1, 0.1)
delta_lambda = 0.1

In [9]:
interact(
    compute_wighted_snr,
    Tg=FloatSlider(value=293.15, min=200, max=500, step=1, description='surrounding ground Temperature (K)'),
    Tf=FloatSlider(value=433.15, min=200, max=500, step=1, description='fumarole Temperature (K)'),
    theta_deg=FloatSlider(value=0, min=0, max=90, step=1, description='Theta (°)'),
    R=FloatSlider(value=460000, min=150000, max=460000, step=1000, description='Distance (m)'),
    tau_a=FloatSlider(value=0.85, min=0.0, max=1.0, step=0.01, description='τₐ (Atmosphere)')
)

interactive(children=(FloatSlider(value=293.15, description='surrounding ground Temperature (K)', max=500.0, m…

<function __main__.compute_wighted_snr(Tg, Tf, theta_deg, R, tau_a)>